# QCArchive - Submitting calculations

This notebook shows the basics of submitting calculations through the client (outside of a dataset)

In [ ]:
import qcportal as ptl
from qcportal.molecules import Molecule

## Create a client object and connect to the demo server

The `PortalClient` is how you interact with the server, including querying records and submitting computations.

The demo server allows for unauthenticated guest access, so no username/password is necessary to read from the server. However, you will
need to log in to submit or modify computations

In [ ]:
client = ptl.PortalClient("https://qcademo.molssi.org", username="YOUR_USERNAME", password="YOUR_PASSWORD")

We are going to submit a singlepoint calculation.

First, we create a molecule (QCElemental molecule). This is a hydrogen molecule with a bond distance of 1.0 bohr.

In [ ]:
mol = Molecule(symbols=['h', 'h'], geometry=[[0.0, 0.0, 0.0, 0.0, 0.0, 1.0]])

Now submit the computation. This sends it to the server, where it is queued for computation. This will eventually be picked up by a worker
if there is a worker that can handle it (proper programs installed, etc).

The methods for adding computations (like `add_singlepoints`) take a list of molecules and the details of how to run the computation. We only have one molecule for now.

* **program** = use this program to run the computation
* **driver** = the main type of computation to run (energy, gradient, properties, etc)
* **method**/**basis** = the computational model to use

All `add_` functions return two objects. The first is metadata about the addition, and second is a list of record ids - we will only have one because we only submitted one molecule.
We can use these IDs later to retrieve the record.

In [ ]:
meta, record_ids = client.add_singlepoints([mol], program='psi4', driver='energy', method='b3lyp', basis='def2-tzvp')

### Metadata

The metadata object returned contains a couple important fields

* **error_description** - An string of any major error that occurred
* **errors** - Per-index error
* **inserted_idx** - Indices (of the molecule list) that were inserted
* **existing_idx** - Indices (of the molecule list) that already existed on the server

The fields in the metadata refer to the index of a particular computation in our input. In this case, it refers to the index
of the molecule in the molecule list, since there is one-to-one correspondence between molecules and computations.

For example, if `inserted_idx=[2, 3]` and `existing_idx=[0, 1, 4]`, that means that records with this specification and molecules `mol[0]`, `mol[1]`, and `mol[4]` already exist
on the server, and `mol[2]` and `mol[3]` are new computations.

In [ ]:
print(meta)

### The IDs

The IDs are returned in the same order as the molecule list we passed in. In this case, we only have one.

In [ ]:
print(record_ids)

Using that ID, we can get the record and view the information as usual

In [ ]:
record = client.get_records(record_ids[0])
print(record.status)